In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Thêm thư viện
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from imutils import paths
from keras.applications.vgg16 import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
import random
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import joblib
import cv2
from sklearn.utils.extmath import softmax
from keras import models
from keras import layers
from keras import optimizers

In [ ]:
!unzip /content/drive/MyDrive/KDL/Train.zip

In [ ]:
# Lấy các đường dẫn đến ảnh.
image_path = list(paths.list_images('Train'))
print(image_path)
# Đổi vị trí ngẫu nhiên các đường dẫn ảnh
random.shuffle(image_path)

In [ ]:
# Đường dẫn ảnh sẽ là dataset/tên_loài_hoa/tên_ảnh ví dụ dataset/Bluebell/image_0241.jpg nên p.split(os.path.sep)[-2] sẽ lấy ra được tên loài hoa
labels = [p.split(os.path.sep)[-2] for p in image_path]

# Chuyển tên các loài hoa thành số
le = LabelEncoder()
labels = le.fit_transform(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected layer ở cuối.
model = VGG16(weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    
    image = np.expand_dims(image, 0)
    #chua hieu
    image = imagenet_utils.preprocess_input(image)
    
    list_image.append(image)
    
list_image = np.vstack(list_image)

In [ ]:
features = model.predict(list_image)

# Giống bước flatten trong CNN, chuyển từ tensor 3 chiều sau ConvNet sang vector 1 chiều
features = features.reshape((features.shape[0], 512*7*7))

In [ ]:
model1 = models.Sequential()
model1.add(layers.Dense(6, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model1.fit(features, labels,epochs=100)

Epoch 1/100
163/163 [==============================] - 1s 4ms/step - loss: 7.5407e-07 - accuracy: 1.0000
Epoch 2/100
163/163 [==============================] - 1s 4ms/step - loss: 7.0692e-08 - accuracy: 1.0000
Epoch 3/100
163/163 [==============================] - 1s 4ms/step - loss: 5.4987e-08 - accuracy: 1.0000
Epoch 4/100
163/163 [==============================] - 1s 4ms/step - loss: 4.4715e-08 - accuracy: 1.0000
Epoch 5/100
163/163 [==============================] - 1s 4ms/step - loss: 3.7126e-08 - accuracy: 1.0000
Epoch 6/100
163/163 [==============================] - 1s 4ms/step - loss: 3.1623e-08 - accuracy: 1.0000
Epoch 7/100
163/163 [==============================] - 1s 4ms/step - loss: 2.7060e-08 - accuracy: 1.0000
Epoch 8/100
163/163 [==============================] - 1s 4ms/step - loss: 2.3368e-08 - accuracy: 1.0000
Epoch 9/100
163/163 [==============================] - 1s 4ms/step - loss: 2.0364e-08 - accuracy: 1.0000
Epoch 10/100
163/163 [==============================] -

In [ ]:
model1.save('my_model.h5')